In [143]:
import gurobipy as gb
import numpy as np
import csv

In [151]:
NUMDISTRICTS = 9
NUMTOWNS = 351

# basic voter data as numpy arrays - POPULATION, DEMOCRATS, REPUBLICANS, INDEPENDENTS
# each town is assigned an index in the idx dict; e.g. to get index of BEDFORD use idx['BEDFORD']
# to get population of BEDFORD use POP[idx['BEDFORD']]
with open("Voter_Data_MA_CSV_Version_2.csv", 'r') as votefile:
    votereader = csv.reader(votefile, delimiter=',')
    next(votereader)
    TOWNS = []
    POP = []
    DEMS = []
    REPS = []
    INDS = []
    idx = {}
    i = 0
    for row in votereader:
        town = row[0]
        TOWNS.append(town)
        idx[town] = i
        i += 1
        POP.append(int(row[-1]))
        DEMS.append(int(row[-3]))
        REPS.append(int(row[-2]))
        INDS.append(POP[-1] - DEMS[-1] - REPS[-1])
    POP = np.array(POP)
    DEMS = np.array(DEMS)
    REPS = np.array(REPS)
    INDS = np.array(INDS)

# pairwise distance data, as a numpy array
# use idx dict to get indices
with open("dist_pairs.csv", 'r') as distfile:
    # pairwise dist
    distreader = csv.reader(distfile, delimiter=',')
    DIST = np.zeros((i,i))
    for row in distreader:
        DIST[idx[row[0]]][idx[row[1]]] = float(row[-1])

# pairwise adjacency matrix (1/0 for adjacent/not adj)
# use idx dict to get indices
with open ('Adjacencies_Fixed.csv', 'r') as adjfile:
    adjreader = csv.reader(adjfile, delimiter=',')
    ADJ = np.zeros((i,i))
    for row in adjreader:
        i1 = idx[row[0]]
        for t2 in row[1:]:
            if not t2 == '':
                ADJ[i1][idx[t2]] = 1

In [152]:
# init model
m = gb.Model()

# min and max population for each district
MINPOP = 720000
MAXPOP = 735000

# big-M (for population of state)
M = 10**6

# total number of voters
VOTERS=sum(REPS)+sum(DEMS)

# init variables
# Dwin_i - whether D wins district i (0/1)
# Rwin_i - whether R wins district i (0/1)
# Ddiff_i - D wasted votes minus R wasted votes in district i (free variable)
# y - variable for removing abs. value from objective (pos float)
# z_t^i - if town t in district i (0/1)

Dwin = []
for i in range(NUMDISTRICTS):
    Dwin.append(m.addVar(name='Dwin'+str(i), vtype=gb.GRB.BINARY))
    
Rwin = []
for i in range(NUMDISTRICTS):
    Rwin.append(m.addVar(name='Rwin'+str(i), vtype=gb.GRB.BINARY))
    
Ddiff = []
for i in range(NUMDISTRICTS):
    Ddiff.append(m.addVar(name='Ddiff'+str(i), lb=-1e31))
    
y = m.addVar(name='y')

z = [[] for t in range(len(idx))]
for t, zt in enumerate(z):
    for i in range(NUMDISTRICTS):
        zt.append(m.addVar(name='z_'+str(t)+'('+str(i)+')', vtype=gb.GRB.BINARY))    

        
# f_ij - amount of flow from town i to town j (pos float)
# sink_i^t - whether district i has a sink at town t (0/1)
f = [[] for t in range(NUMTOWNS)]
for t, ft in enumerate(f):
    for i in range(NUMTOWNS):
        ft.append(m.addVar(name='f_'+str(t)+'('+str(i)+')'))
        
sink = [[] for i in range(NUMDISTRICTS)]
for i, sinki in enumerate(sink):
    for t in range(NUMTOWNS):
        sinki.append(m.addVar(name='sink_'+str(i)+'('+str(t)+')', vtype=gb.GRB.BINARY))   

                
m.update()

# init constraints
# each town is in exactly one district
# sum_i z_t^i = 1
for t in range(len(TOWNS)):
    m.addConstr(sum(z[t]), sense=gb.GRB.EQUAL, rhs=1., name='towndist'+str(t))
    
# district population constraint:
# MINPOP <= sum_t z_t^i * POP_i <= MAXPOP for all districts i
for i in range(NUMDISTRICTS):
    m.addConstr(sum([z[t][i]*POP[t] for t in range(len(TOWNS))]), sense=gb.GRB.LESS_EQUAL, rhs=MAXPOP, name='maxpop'+str(i))
    m.addConstr(sum([z[t][i]*POP[t] for t in range(len(TOWNS))]), sense=gb.GRB.GREATER_EQUAL, rhs=MINPOP, name='minpop'+str(i))    
    
# constraints to define Dwin and Rwin
for i in range(NUMDISTRICTS):
    m.addConstr(Dwin[i]+Rwin[i], sense=gb.GRB.EQUAL, rhs=1., name='onewins'+str(i))
    m.addConstr(sum([z[t][i]*(DEMS[t]-REPS[t]) for t in range(len(TOWNS))])-M*Dwin[i], sense=gb.GRB.LESS_EQUAL, rhs=0., name='checkDwins'+str(i))
    m.addConstr(sum([z[t][i]*(REPS[t]-DEMS[t]) for t in range(len(TOWNS))])-M*Rwin[i], sense=gb.GRB.LESS_EQUAL, rhs=0., name='checkRwins'+str(i))

# constraints to find number of wasted votes
for i in range(NUMDISTRICTS):
    m.addConstr(sum([z[t][i]*(DEMS[t]-3*REPS[t]) for t in range(len(TOWNS))])/2+2*M*Rwin[i]-Ddiff[i], sense=gb.GRB.GREATER_EQUAL, rhs=0., name='wastedgap1'+str(i))
    m.addConstr(sum([z[t][i]*(DEMS[t]-3*REPS[t]) for t in range(len(TOWNS))])/2-2*M*Rwin[i]-Ddiff[i], sense=gb.GRB.LESS_EQUAL, rhs=0., name='wastedgap2'+str(i))
    m.addConstr(sum([z[t][i]*(3*DEMS[t]-REPS[t]) for t in range(len(TOWNS))])/2+2*M*Dwin[i]-Ddiff[i], sense=gb.GRB.GREATER_EQUAL, rhs=0., name='wastedgap3'+str(i))
    m.addConstr(sum([z[t][i]*(3*DEMS[t]-REPS[t]) for t in range(len(TOWNS))])/2-2*M*Dwin[i]-Ddiff[i], sense=gb.GRB.LESS_EQUAL, rhs=0., name='wastedgap4'+str(i))

# constraints to find z (efficiency gap in # of votes)
m.addConstr(sum(Ddiff)-y, sense=gb.GRB.LESS_EQUAL, rhs=0., name='Egap1')
m.addConstr(-sum(Ddiff)-y, sense=gb.GRB.LESS_EQUAL, rhs=0., name='Egap2')


# Force districts to be contiguous using flow
# Only one sink per district
for i in range(NUMDISTRICTS):
    m.addConstr(sum(sink[i]), sense=gb.GRB.EQUAL, rhs=1., name='onesink'+str(i))
    
# No flow from outside region and inflow does not exceeed M-1 (big-M = most towns in a district = NUMTOWNS)
for j in range(NUMDISTRICTS):
    for t in range(NUMTOWNS):
        m.addConstr(sum([f[t][t2] for t2 in np.where(ADJ[t]==1)[0]])+(1-NUMTOWNS)*z[t][j], sense=gb.GRB.LESS_EQUAL, rhs=0., name='flow'+str(j)+str(t))

# Net outflow
for j in range(NUMDISTRICTS):
    for t in range(NUMTOWNS):
        m.addConstr(sum([f[t][t2] for t2 in np.where(ADJ[t]==1)[0]])-sum([f[t3][t] for t3 in np.where(ADJ[t]==1)[0]])-z[t][j]+NUMTOWNS*sink[j][t], sense=gb.GRB.GREATER_EQUAL, rhs=0., name='flowconstraint'+str(j)+str(t))


m.update()


# objective
print('INITIALIZING OBJECTIVE')
# min y/sum_i(r_i+d_i) (towns i)
m.update()
m.setObjective(y/VOTERS, gb.GRB.MINIMIZE)
print('SOLVING')
m.optimize()


INITIALIZING OBJECTIVE
SOLVING
Optimize a model with 6761 rows, 129547 columns and 89705 nonzeros
Variable types: 123211 continuous, 6336 integer (6336 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+06]
  Objective range  [2e-07, 2e-07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+05]
Presolve removed 9 rows and 121780 columns
Presolve time: 0.49s

Explored 0 nodes (0 simplex iterations) in 0.56 seconds
Thread count was 1 (of 4 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


In [109]:
# Code for looking at some results

#print([ri.x for ri in r]) # Print values of r
print(Ddiff) # Print wasted vote differences in each district
print(y)
demarray=np.zeros(9)
reparray=np.zeros(9)
for i in range(NUMDISTRICTS):
    for t in range(len(TOWNS)):
        demarray[i]+=z[t][i].x*DEMS[t]
for i in range(NUMDISTRICTS):
    for t in range(len(TOWNS)):
        reparray[i]+=z[t][i].x*REPS[t]
print(demarray) # Print number of Democrats in each district
print(reparray) # Print number of Republicans in each district
print(sum(REPS)+sum(DEMS))

# Print names of towns in each district
for number in range(9):
    print(number+1)
    district=np.zeros(351)
    for i in range(351):
        district[i]=z[i][number].x
    print(district) # Print vector indicating which towns are in district "number+1"
    for i in range(351):
        if district[i]==1:
            print(("'"+str(TOWNS[i])+"',").strip()) # Print list of towns in district "number+1"
            
    

        

[<gurobi.Var Ddiff0 (value -65208.49999994044)>, <gurobi.Var Ddiff1 (value 137280.0000000612)>, <gurobi.Var Ddiff2 (value -100855.49999992728)>, <gurobi.Var Ddiff3 (value -8528.999999923763)>, <gurobi.Var Ddiff4 (value -1790.999999926311)>, <gurobi.Var Ddiff5 (value -9329.999999922748)>, <gurobi.Var Ddiff6 (value 90341.00000009124)>, <gurobi.Var Ddiff7 (value -27611.49999993856)>, <gurobi.Var Ddiff8 (value -80767.49999993554)>]
<gurobi.Var y (value 66471.99999936219)>
[324578. 435342. 354351. 368388. 391209. 383433. 416074. 346861. 331833.]
[151665.  53594. 185354. 128482. 131597. 134031.  78464. 134028. 164456.]
4513740
1
[-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.  1. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.  1. -0.
  1. -0. -0. -0. -0. -0. -0.  1. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0.  1.  1. -0. -0. -0. -0. -0.  1. -0. -0. -0. -0. -0. -0.
 -0.

In [100]:
ADJ[0][1]
a=2*np.arange(20)
a[np.where(ADJ[0]==1)[0]]
type(f[0])
#f[0][np.where(ADJ[0]==1)[0].astype(int)]


sum(np.asarray(f)[0][np.where(ADJ[0]==1)])
sum([f[0][j] for j in np.where(ADJ[0]==1)[0]])
#ADJ[0][np.where(ADJ[0]==1)]

#sum([f[0][j] for j in np.where(ADJ[0]==1)])
#sum([f[0][j] for j in np.where(ADJ[0][j]==1)])

<gurobi.LinExpr: f_0(8) + f_0(11) + f_0(14)>

In [89]:
sum(f[0][8:11])

<gurobi.LinExpr: f_0(8) + f_0(9) + f_0(10)>

In [96]:
sum([a[j] for j in np.where(ADJ[0]==1)[0]])

66

In [131]:
t=15
sum([f[t][j] for j in np.where(ADJ[t]==1)[0]])

<gurobi.LinExpr: f_15(18) + f_15(22) + f_15(31) + f_15(33) + f_15(39) + f_15(45)>

In [155]:
[TOWNS[j] for j in np.where(sum(ADJ!=np.transpose(ADJ)))[0]]

[]